In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from KalmanFilter import KalmanFilter
import math
from filterpy.kalman import KalmanFilter


#model = YOLO('C:/Users/loren/Desktop/TennisBounceDetector/models/yoloV5_2k.pt')

In [ ]:
import os

dir = 'C:/Users/loren/Desktop/TennisBounceDetector/Dataset/game1/Clip6/'

for clip in os.listdir(dir):
    f = os.path.join(dir, clip)
    result = model.track(dir + clip, save=True)

In [ ]:
import cv2
import os

video_dim = (1280, 720)
fps = 25

vidwriter = cv2.VideoWriter("outputTrack3.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, video_dim)

dir = 'C:/Users/loren/Desktop/tennis/game1/Clip1'
frames = []
for clip in os.listdir(dir):
    f = os.path.join(dir, clip)
    img = cv2.imread(f, cv2.IMREAD_COLOR)
    vidwriter.write(img)
    
vidwriter.release()

# Tracknet

In [ ]:
import numpy as np
import pandas as pd
import sys, getopt
import os
from glob import glob
#from keras.preprocessing.image import ImageDataGenerator 
#from tensorflow.keras.utils import array_to_img
#from tensorflow.keras.utils import img_to_array
#from tensorflow.keras.utils import load_img
import pandas as pd
# from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
import keras.backend as K
from keras import optimizers
from keras.activations import *
import tensorflow as tf
import gc
import itertools
import cv2
import math
import matplotlib.pyplot as plt
from sys import getsizeof

In [ ]:
print(tf.__version__)

In [ ]:
#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)

model = tf.keras.models.load_model('models/model906_30', custom_objects={'custom_loss':custom_loss})

In [ ]:
#ADADELTA = optimizers.Adadelta(learning_rate=lr) # 1e-5 if fine tuning
model.compile(loss=custom_loss) #,optimizer=ADADELTA)

In [ ]:
model.input_shape

In [ ]:
import os

dir = 'C:/Users/loren/Desktop/TennisBounceDetector/Dataset/game1/Clip1/'
result = []

for clip in os.listdir(dir):
    img = tf.keras.utils.load_img(os.path.join(dir, clip))
    model.predict(tf.convert_to_tensor(img, dtype=tf.int8))

In [ ]:
video_dim = (1280, 720)
fps = 25

vidwriter = cv2.VideoWriter("outputTrack3.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, video_dim)

dir = 'C:/Users/loren/Desktop/TennisBounceDetector/runs/detect/predict5'
frames = []
for clip in os.listdir(dir):
    f = os.path.join(dir, clip)
    img = cv2.imread(f, cv2.IMREAD_COLOR)
    vidwriter.write(img)
    
vidwriter.release()

# Kalman filtering

In [ ]:
df = pd.read_csv('./outcsv/veryshorttest.csv')

In [ ]:
df.head()

In [ ]:
VideoCap = cv2.VideoCapture('./videoin/veryshorttest.mp4')
#skip first 3 frames
for i in range(3):
    ret, frame = VideoCap.read()

In [ ]:
 #Variable used to control the speed of reading the video
ControlSpeedVar = 70  #Lowest: 1 - Highest:100

HiSpeed = 100


In [ ]:
# KF = KalmanFilter(0.1, 1, 1, 1, 0.1,0.1)
# discard first frame 
#ret, frame = VideoCap.read()
i = 0


In [ ]:
frames_df = df.iloc[0:3]
# get x and y in a 2d array
measurements = frames_df.to_numpy()
measurements = measurements[:, 2:4]
measurements 

In [ ]:
from cv2 import KalmanFilter
# Collect measurements (example values)
#measurements = np.array([[1, 2], [3, 4], [5, 6]])


# Calculate mean and covariance of measurements
mean_meas = np.mean(measurements, axis=0)
cov_meas = np.cov(measurements, rowvar=False)

# Initialize the Kalman filter
kf = KalmanFilter(4, 2)  # 4 state variables, 2 measurements

# Set initial mean
kf.statePost = np.array([mean_meas[0], mean_meas[1], 0, 0], dtype=np.float32)

# Set initial covariance
kf.errorCovPost = cov_meas.astype(np.float32)
# Initialize the Kalman filter
kf = KalmanFilter(4, 2) # 4 state variables, 2 measurements
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kf.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 1e-4
kf.measurementNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 1e-1

In [ ]:
# from the 3 measurements calculate velocity
vx = (measurements[2][0]- measurements[0][0])/2
vy = (measurements[2][1]- measurements[0][1])/2

In [ ]:
f = KalmanFilter (dim_x=4, dim_z=2)
f.x = np.array([mean_meas,    # position
                [vx, vy]])   # velocity

In [ ]:
# state transition matrix
f.F = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
f.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
f.P = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 1e-4
f.R =np.array([[1, 0], [0, 1]], np.float32) * 1e-1


In [ ]:
while(True):
    ret, frame = VideoCap.read()
    # get first row of dataset
    _, _,c_x, c_y = df.iloc[i]
    
    
    if (not math.isnan(c_x)) and (not math.isnan(c_y)):
        f.update(np.array([c_x, c_y], np.float32).reshape(2, 1))  
    predicted = f.predict()

        # Draw the predicted position
    cv2.circle(frame, (int(predicted[0]), int(predicted[1])), 10, (0, 255, 0), -1)
    cv2.imshow('image', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
            VideoCap.release()
            cv2.destroyAllWindows()
            break

    cv2.waitKey(HiSpeed-ControlSpeedVar+1)
    i= i+1
    
    